# Introduction

# Datasets

The data was obtained from [the Broad Institute](http://portals.broadinstitute.org/cgi-bin/cancer/publications/view/43) and is stored as follows:

<table border="2" cellspacing="0" cellpadding="6" rules="groups" frame="hsides">


<colgroup>
<col  class="org-left" />

<col  class="org-left" />
</colgroup>
<thead>
<tr>
<th scope="col" class="org-left">Type of data</th>
<th scope="col" class="org-left">File name</th>
</tr>
</thead>

<tbody>
<tr>
<td class="org-left">Training data</td>
<td class="org-left">`data_set_ALL_AML_train.txt`</td>
</tr>


<tr>
<td class="org-left">Training data class labels</td>
<td class="org-left">`ALL_vs_AML_train_set_38_sorted.cls`</td>
</tr>


<tr>
<td class="org-left">Testing data</td>
<td class="org-left">`data_set_ALL_AML_independent.txt`</td>
</tr>


<tr>
<td class="org-left">Testing data class labels</td>
<td class="org-left">`Leuk_ALL_AML.test.cls`</td>
</tr>
</tbody>
</table>



# Preparing the data

In [1]:
datasets = {}

In [2]:
def clean_training_data():
    clean_lines = []
    with open("data_set_ALL_AML_train.txt", "r") as f:
        lines = f.readlines()
        clean_lines = [l.rstrip('\t\n') for l in lines]

    with open("data_set_ALL_AML_train_cleaned.txt", "w") as f:
        f.writelines('\n'.join(clean_lines))

clean_training_data()

In [31]:
import numpy, scipy, pandas
from pandas import DataFrame as df
import sklearn
import re

def load_data(X_filename, y_filename):
    df_X = pandas.read_csv(X_filename, sep="\t")
    df_X = df_X.select(lambda x: not re.search('call\.*', x), axis=1)
    df_X = df_X.drop(['Gene Description', 
                      'Gene Accession Number'], axis=1)
    df_X = df_X.T
    X = df_X.values

    with open(y_filename, "r") as fin:
        data = fin.read().splitlines(True)
    data = data[1].rstrip()

    y = numpy.fromstring(data, sep=" ")

    return X, y


X_train, y_train = load_data("data_set_ALL_AML_train_cleaned.txt",
                             "ALL_vs_AML_train_set_38_sorted.cls")
X_test, y_test = load_data("data_set_ALL_AML_independent.txt",
                           "Leuk_ALL_AML.test.cls")
y_test = y_test[1:]  # dataset has one additional 0 at beginning, 
                     # so remove it
datasets['leukemia'] = [X_train,X_test,y_train,y_test]

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:8: FutureWarning: 'select' is deprecated and will be removed in a future release. You can use .loc[labels.map(crit)] as a replacement
  


In [7]:
# MNIST dataset
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
mnist = fetch_mldata('MNIST original')
X = df(mnist.data)
y = df(mnist.target)
X['label'] = y
X.head()
X = X[(X['label']==0.0) | (X['label']==1.0)]
datasets['mnist'] = train_test_split(X.drop('label',1),X['label'], test_size=0.2, random_state=42)

# Experiment

## Logistic Regression
To choose the $\gamma$ function of the RBF kernel (where $\gamma = 1/(2\sigma^2)$) we follow the heuristic choice mentioned in Gretton et al. (p. 748) of setting $\sigma$ to equal the median distance between points of the training data.


In [35]:
import sklearn.linear_model
import sklearn.kernel_ridge
import sklearn.metrics.pairwise
from sklearn.metrics import confusion_matrix
from sklearn.metrics import zero_one_loss
from scipy.spatial.distance import cdist
import statistics

def estimate_log_regression(X_train, X_test, y_train, y_test, 
                            kernelize=False, penalty='l2'):
    if kernelize == True:      
        # Calculate gamma as in Gretton et al.
        b = cdist(X_train, X_train).ravel()
        gamma = 1/(2 * pow(statistics.median(b), 2))
        # Transform data via RBF kernel 
        K_train = sklearn.metrics.pairwise.rbf_kernel(X_train, X_train, gamma=gamma)
        X_test = sklearn.metrics.pairwise.rbf_kernel(X_test, X_train, gamma=gamma)
        X_train = K_train
    # Fit logistic regression
    clf = sklearn.linear_model.LogisticRegression(penalty=penalty)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    err = zero_one_loss(y_test, y_pred)
    conf_mat = confusion_matrix(y_test, y_pred)
    return {'error' : err, 'confusion' : conf_mat}


## SVM



In [36]:
from sklearn.svm import SVC
#from sklearn.cross_validation import StratifiedKFold
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# This code is a modification of code at
# http://ogrisel.github.io/scikit-learn.org/sklearn-tutorial/auto_examples/svm/plot_svm_parameters_selection.html

def find_svm_best_params(kernel_type):
    C_range = 2. ** numpy.arange(-5, 15, 2)
    gamma_range = 2. ** numpy.arange(-5, 3, 2)
    param_grid = dict(gamma=gamma_range, C=C_range)
    grid_search = GridSearchCV(SVC(kernel=kernel_type), param_grid, cv=5)
    grid_search.fit(X_train, y_train)
    bestparams = grid_search.best_params_
    return bestparams

def estimate_svm(X_train, X_test, y_train, y_test, kernel_type):
    bestparams = find_svm_best_params(kernel_type)
    our_svm = SVC(kernel=kernel_type, C=bestparams['C'], gamma=bestparams['gamma'])
    our_svm.fit(X_train, y_train)
    y_pred = our_svm.predict(X_test)
    err = zero_one_loss(y_test, y_pred)
    conf_mat = confusion_matrix(y_test, y_pred)
    return {'error' : err, 'confusion' : conf_mat}

## Compute Results

In [40]:
def evaluate_models(X_train, X_test, y_train, y_test):
    return {
        'logistic_l1' : estimate_log_regression(X_train, X_test, y_train, y_test, 
                                                penalty='l1'),
        'logistic_l2' : estimate_log_regression(X_train, X_test, y_train, y_test),
        'logistic_l1_rbf' : estimate_log_regression(X_train, X_test, y_train, y_test,  
                                                penalty='l1', kernelize=True),
        'logistic_l2_rbf' : estimate_log_regression(X_train, X_test, y_train, y_test, 
                                               kernelize=True),
        'svm_rbf' : estimate_svm(X_train, X_test, y_train, y_test,  'rbf'),
        'svm_linear' : estimate_svm(X_train, X_test, y_train, y_test,  'linear')}


## Print results



In [38]:
def display_results(models):
    series_index = ["Model", "Empirical error"]
    results_df = df(columns=('Model', 'Empirical error'))
    for m in models:
        results_df = results_df.append(pandas.Series([m,models[m]['error']],index=series_index), 
                          ignore_index=True)
    display(results_df.sort_values('Model'))

In [41]:
print("Leukemia Results")
leukemia_models = evaluate_models(*datasets['leukemia'])
display_results(leukemia_models)

Leukemia Results


,Model,Empirical error
5,logistic_l1,0.000000
0,logistic_l1_rbf,0.411765
1,logistic_l2,0.029412
4,logistic_l2_rbf,0.352941
2,svm_linear,0.029412
3,svm_rbf,0.411765


In [42]:
print("MNIST Results")
mnist_models = evaluate_models(*datasets['mnist'])
display_results(mnist_models)

MNIST Results


,Model,Empirical error
5,logistic_l1,0.001015
0,logistic_l1_rbf,0.001691
1,logistic_l2,0.002030
4,logistic_l2_rbf,0.000677
2,svm_linear,0.001691
3,svm_rbf,0.468539



# References (move to separate file later)



Gretton, Arthur et al. 2012. "A Kernel Two-Sample Test." *Journal of Machine Learning Research*. Vol 13, p. 723-773.

Hsu, Chih-Wei et al. 2016. "A Practical Guide to Support Vector Classification." Department of Computer Science, National Taiwan University.

